https://github.com/Arlens26/Tokenizador_Y_Similaridad/blob/84fe73596e4e061d6bfcd43a231fd4af544c8f07/crf.py

https://github.com/l11x0m7/book-of-qna-code/blob/17c218f754de14fee30d54ea36584e9e5c14f1e6/ch2/crf/crf_sample.py

https://github.com/soroush-ziaeinejad/NLP_Course/blob/bae592ca012a77729c48b30720978148e584563d/ProposedMethod_ziaeines/src/CRF.py

In [1]:
!pip install scikit-learn==0.22.2

## Exemplo

## Imports

In [2]:
#!git clone -s https://github.com/mstauffer/tcdf_text_classification.git
from tcdf_text_classification.iob_transformer import iob_transformer

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/alilim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-11-28 01:40:42.280516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 01:40:42.413644: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-28 01:40:42.442582: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 01:40:42.968523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

## Dataset

In [6]:
import pandas as pd

df = pd.read_csv('DODFCorpus_contratos_licitacoes_v2.csv')

df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1)

df['texto'] = df['texto'].str.replace(r'([A-Za-z]:)[0-9]', r'\1 ', regex=True)


In [7]:
df = df.query("tipo_rel == 'REL_EXTRATO_CONVENIO' | tipo_rel == 'REL_EXTRATO_CONTRATO'")

In [8]:
iob_v = iob_transformer('id_ato', 'texto', 'tipo_ent', keep_punctuation=True, return_df=False)

acts, labels = iob_v.transform(df)

In [9]:
tags = set()

for label in labels:
    for tag in label:
        tags.add(tag)
tags = list(tags)
tags_amt = len(tags)

tags_amt

51

In [10]:
def remove_wrong_tags(label_list):
  for label in label_list:
    for idx,w in enumerate(label):
      if w in ['B-11','B-12','B-50']:
        label[idx] = 'O'

In [11]:
remove_wrong_tags(labels)

In [12]:
def word2features(sent, i):
    word = sent[i]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),        
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [13]:
inputs = [sent2features(s) for s in acts]

In [14]:
targets = labels

In [15]:
from sklearn.model_selection import KFold

In [16]:
kfold = KFold(n_splits=5, shuffle=True)

In [17]:
from seqeval.metrics import f1_score, classification_report, precision_score, recall_score, accuracy_score

In [18]:
import numpy as np

In [19]:
f1 = []
acc = []
reps = []

for train, test in kfold.split(inputs, targets):

    from sklearn_crfsuite import CRF

    x_train = np.array(inputs)[train.astype(int)]
    x_test = np.array(inputs)[test.astype(int)]

    y_train = np.array(targets)[train.astype(int)]
    y_test = np.array(targets)[test.astype(int)]

    crf = CRF(
    algorithm='lbfgs', 
    c1=0.42194423530178, 
    c2=0.044686453706365134, 
    max_iterations=70, 
    all_possible_transitions=True, verbose=1)

    crf.fit(X=x_train, y=y_train)

    y_pred = crf.predict(x_test)

    f1.append(f1_score(y_test, y_pred))

    reps.append(classification_report(y_test,y_pred))

    acc.append(accuracy_score(y_test, y_pred))



/tmp/ipykernel_3943207/2053661115.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(inputs)[train.astype(int)]
/tmp/ipykernel_3943207/2053661115.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test = np.array(inputs)[test.astype(int)]
/tmp/ipykernel_3943207/2053661115.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 87343
Seconds required: 0.484

L-BFGS optimization
c1: 0.421944
c2: 0.044686
num_memories: 6
max_iterations: 70
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.48  loss=979333.40 active=86177 feature_norm=1.00
Iter 2   time=6.98  loss=916419.13 active=85927 feature_norm=6.64
Iter 3   time=1.69  loss=559496.60 active=81059 feature_norm=5.45
Iter 4   time=16.77 loss=327749.96 active=84663 feature_norm=5.21
Iter 5   time=10.57 loss=313691.47 active=83875 feature_norm=5.27
Iter 6   time=5.23  loss=300273.38 active=83865 feature_norm=4.49
Iter 7   time=1.74  loss=263350.35 active=72625 feature_norm=5.10
Iter 8   time=5.33  loss=228567.76 active=72268 feature_norm=5.98
Iter 9   time=7.15  loss=212785.43 active=72686 feature_norm=6.39
Iter 1

/home/alilim/.local/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
loading training data to CRFsuite: 100%|██████████| 1413/1413 [00:02<00:00, 545.16it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 89015
Seconds required: 0.518

L-BFGS optimization
c1: 0.421944
c2: 0.044686
num_memories: 6
max_iterations: 70
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.83  loss=999467.98 active=87800 feature_norm=1.00
Iter 2   time=7.65  loss=947051.34 active=87539 feature_norm=6.80
Iter 3   time=1.91  loss=575833.10 active=82706 feature_norm=5.58
Iter 4   time=17.32 loss=347598.77 active=84636 feature_norm=5.33
Iter 5   time=7.54  loss=347134.41 active=83872 feature_norm=4.84
Iter 6   time=7.46  loss=305853.25 active=84560 feature_norm=4.46
Iter 7   time=3.72  loss=303454.30 active=77696 feature_norm=5.83
Iter 8   time=9.25  loss=298556.22 active=77846 feature_norm=5.91
Iter 9   time=5.57  loss=290519.38 active=77976 feature_norm=5.82
Iter 1

loading training data to CRFsuite: 100%|██████████| 1413/1413 [00:02<00:00, 588.63it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 86642
Seconds required: 0.460

L-BFGS optimization
c1: 0.421944
c2: 0.044686
num_memories: 6
max_iterations: 70
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.37  loss=981390.63 active=85453 feature_norm=1.00
Iter 2   time=6.76  loss=918798.21 active=85228 feature_norm=6.62
Iter 3   time=1.69  loss=563599.05 active=80748 feature_norm=5.41
Iter 4   time=15.28 loss=340504.12 active=84296 feature_norm=5.17
Iter 5   time=10.22 loss=323919.04 active=80640 feature_norm=5.34
Iter 6   time=6.74  loss=317587.49 active=82419 feature_norm=5.15
Iter 7   time=6.76  loss=304529.68 active=82746 feature_norm=4.98
Iter 8   time=6.80  loss=301022.46 active=82747 feature_norm=4.89
Iter 9   time=5.06  loss=299714.93 active=82668 feature_norm=4.82
Iter 1

loading training data to CRFsuite: 100%|██████████| 1413/1413 [00:02<00:00, 580.30it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 87508
Seconds required: 0.460

L-BFGS optimization
c1: 0.421944
c2: 0.044686
num_memories: 6
max_iterations: 70
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.43  loss=993695.81 active=86317 feature_norm=1.00
Iter 2   time=6.85  loss=940358.00 active=86061 feature_norm=6.78
Iter 3   time=1.72  loss=568108.27 active=81282 feature_norm=5.57
Iter 4   time=21.77 loss=330941.15 active=83863 feature_norm=5.33
Iter 5   time=8.55  loss=321500.99 active=85287 feature_norm=5.12
Iter 6   time=8.60  loss=307116.23 active=85895 feature_norm=4.97
Iter 7   time=5.13  loss=303983.41 active=85521 feature_norm=4.66
Iter 8   time=1.71  loss=284159.33 active=75987 feature_norm=4.96
Iter 9   time=1.71  loss=245716.39 active=74103 feature_norm=7.04
Iter 1

loading training data to CRFsuite: 100%|██████████| 1413/1413 [00:02<00:00, 585.14it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 88630
Seconds required: 0.456

L-BFGS optimization
c1: 0.421944
c2: 0.044686
num_memories: 6
max_iterations: 70
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=3.45  loss=986292.06 active=87361 feature_norm=1.00
Iter 2   time=6.95  loss=936175.10 active=87143 feature_norm=6.76
Iter 3   time=1.75  loss=572013.90 active=82396 feature_norm=5.52
Iter 4   time=22.33 loss=316865.23 active=84754 feature_norm=5.26
Iter 5   time=10.36 loss=315477.65 active=85490 feature_norm=5.18
Iter 6   time=8.63  loss=310959.56 active=85816 feature_norm=5.06
Iter 7   time=6.95  loss=310933.16 active=85957 feature_norm=4.91
Iter 8   time=5.24  loss=310348.62 active=85947 feature_norm=4.69
Iter 9   time=1.77  loss=298494.11 active=83456 feature_norm=4.68
Iter 1

In [20]:
f1

[0.8154056517775752,
 0.7757211538461538,
 0.8178070898598516,
 0.8170426065162907,
 0.8385308385308385]

In [21]:
np.mean(f1)

0.8129014681061421

In [22]:
np.std(f1)

0.02042902584471683

In [30]:
print(reps[0])
print(reps[1])
print(reps[2])
print(reps[3])
print(reps[4])

                          precision    recall  f1-score   support

cnpj_entidade_contratada       0.74      0.66      0.70        99
cnpj_entidade_convenente       0.00      0.00      0.00         2
   cnpj_orgao_concedente       0.00      0.00      0.00         3
  cnpj_orgao_contratante       0.83      0.63      0.72        30
            codigo_siggo       0.74      0.78      0.76        40
data_assinatura_contrato       0.72      0.85      0.78       243
data_assinatura_convenio       0.00      0.00      0.00         8
     entidade_contratada       0.79      0.69      0.74       369
     entidade_convenente       0.00      0.00      0.00        11
           fonte_recurso       0.98      0.89      0.93       296
        natureza_despesa       0.89      0.85      0.87       244
        nome_responsavel       0.94      0.69      0.79        42
            nota_empenho       0.92      0.79      0.85       281
         numero_contrato       0.86      0.90      0.88       367
         

## Valores do kfold (contratos e aditamentos)

In [28]:
# avaliação nos extratos de contrato

#np.mean(f1)
# 0.8659178331585456

#np.std(f1)
#0.00676026225068485

#f1 
# [0.8724496511780966,
#  0.8605144523998941,
#  0.8555481727574752,
#  0.8723293381969777,
#  0.8687475512602846]

#acc
# [0.9655420835145716,
#  0.9584223925503383,
#  0.9588658202500313,
#  0.964885026101804,
#  0.9619780829518657]

In [29]:
# avaliação nos aditamentos

#np.mean(f1)
# 0.8103676653053375

#np.std(f1)
#0.01321410709546104

#f1 
# [0.8037572988068037,
#  0.8195469585136168,
#  0.8264504687104129,
#  0.813541933829187,
#  0.7885416666666667]

#acc
# [0.9419161589998118,
#  0.9285171627874853,
#  0.9467657211466306,
#  0.9436907582938389,
#  0.9197376733551939]
